In [1]:
!pip install pyDOE

In [2]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
import time
import matplotlib.pyplot as plt

class PhysicsInformedNN:
    def __init__(self, x0, u0, v0, tb, X_f, layers, lb, ub):
        self.lb = lb
        self.ub = ub

        self.x0 = torch.tensor(x0, dtype=torch.float32, requires_grad=True)
        self.t0 = torch.tensor(np.zeros_like(x0), dtype=torch.float32, requires_grad=True)
        self.u0 = torch.tensor(u0, dtype=torch.float32)
        self.v0 = torch.tensor(v0, dtype=torch.float32)

        self.x_lb = torch.tensor(np.zeros_like(tb) + lb[0], dtype=torch.float32, requires_grad=True)
        self.t_lb = torch.tensor(tb, dtype=torch.float32, requires_grad=True)

        self.x_ub = torch.tensor(np.zeros_like(tb) + ub[0], dtype=torch.float32, requires_grad=True)
        self.t_ub = torch.tensor(tb, dtype=torch.float32, requires_grad=True)

        self.x_f = torch.tensor(X_f[:, 0:1], dtype=torch.float32, requires_grad=True)
        self.t_f = torch.tensor(X_f[:, 1:2], dtype=torch.float32, requires_grad=True)

        # Initialize the neural network
        self.model = self.build_model(layers)
        self.optimizer = torch.optim.Adam(self.model.parameters())

    def build_model(self, layers):
        model = []
        num_layers = len(layers)
        for i in range(num_layers - 2):
            model.append(nn.Linear(layers[i], layers[i+1]))
            model.append(nn.Tanh())
        model.append(nn.Linear(layers[-2], layers[-1]))
        return nn.Sequential(*model)

    def forward_uv(self, x, t):
        X = torch.cat([x, t], dim=1)
        uv = self.model(X)
        u = uv[:, 0:1]
        v = uv[:, 1:2]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        return u, v, u_x, v_x

    def net_f_uv(self, x, t):
        u, v, u_x, v_x = self.forward_uv(x, t)
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
        f_u = u_t + 0.5 * v_xx + (u**2 + v**2) * v
        f_v = v_t - 0.5 * u_xx - (u**2 + v**2) * u
        return f_u, f_v

    def loss_func(self):
        u0_pred, v0_pred, _, _ = self.forward_uv(self.x0, self.t0)
        u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred = self.forward_uv(self.x_lb, self.t_lb)
        u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred = self.forward_uv(self.x_ub, self.t_ub)
        f_u_pred, f_v_pred = self.net_f_uv(self.x_f, self.t_f)

        loss = torch.mean((self.u0 - u0_pred) ** 2) + \
               torch.mean((self.v0 - v0_pred) ** 2) + \
               torch.mean((u_lb_pred - u_ub_pred) ** 2) + \
               torch.mean((v_lb_pred - v_ub_pred) ** 2) + \
               torch.mean((u_x_lb_pred - u_x_ub_pred) ** 2) + \
               torch.mean((v_x_lb_pred - v_x_ub_pred) ** 2) + \
               torch.mean(f_u_pred ** 2) + \
               torch.mean(f_v_pred ** 2)
        return loss

    def train(self, n_iter):
        for it in range(n_iter):
            self.optimizer.zero_grad()
            loss = self.loss_func()
            loss.backward()
            self.optimizer.step()
            if it % 10 == 0:
                print(f'Iter: {it}, Loss: {loss.item():.3e}')

    def predict(self, X_star):
        x_star = torch.tensor(X_star[:, 0:1], dtype=torch.float32)
        t_star = torch.tensor(X_star[:, 1:2], dtype=torch.float32)

        with torch.no_grad():
            u_pred, v_pred, _, _ = self.forward_uv(x_star, t_star)
            f_u_pred, f_v_pred = self.net_f_uv(x_star, t_star)

        return u_pred.numpy(), v_pred.numpy(), f_u_pred.numpy(), f_v_pred.numpy()


In [5]:
# Load data from NLS.mat
data = scipy.io.loadmat('NLS.mat')
t = data['tt'].flatten()[:, None]       # time points
x = data['x'].flatten()[:, None]        # spatial points
Exact = data['uu']                      # complex solution

# Separate real and imaginary parts of the solution
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)

# Define bounds, layer configuration, and number of samples
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])
layers = [2, 100, 100, 100, 100, 2]
N0 = 50
N_b = 50
N_f = 20000

# Sampling initial conditions
idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0 = x[idx_x, :]
u0 = Exact_u[idx_x, 0:1]
v0 = Exact_v[idx_x, 0:1]

# Sampling boundary conditions
idx_t = np.random.choice(t.shape[0], N_b, replace=False)
tb = t[idx_t, :]

# Collocation points
X_f = lb + (ub - lb) * lhs(2, N_f)


In [6]:
# Initialize and train model
model = PhysicsInformedNN(x0, u0, v0, tb, X_f, layers, lb, ub)
start_time = time.time()
model.train(5000)
elapsed = time.time() - start_time
print(f'Training time: {elapsed:.4f} seconds')

# Prediction
X_star = np.hstack((np.repeat(x, len(t)), np.tile(t, len(x)))).reshape(-1, 2)
u_pred, v_pred, f_u_pred, f_v_pred = model.predict(X_star)

# Compute and display errors
u_star = Exact_u.T.flatten()[:, None]
v_star = Exact_v.T.flatten()[:, None]
error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
error_v = np.linalg.norm(v_star - v_pred, 2) / np.linalg.norm(v_star, 2)
print(f'Error u: {error_u:.3e}')
print(f'Error v: {error_v:.3e}')

Iter: 0, Loss: 1.040e+00
Iter: 10, Loss: 3.152e-01
Iter: 20, Loss: 2.466e-01
Iter: 30, Loss: 1.674e-01
Iter: 40, Loss: 1.394e-01
Iter: 50, Loss: 1.281e-01
Iter: 60, Loss: 1.170e-01
Iter: 70, Loss: 1.121e-01
Iter: 80, Loss: 1.068e-01
Iter: 90, Loss: 1.026e-01
Iter: 100, Loss: 9.679e-02
Iter: 110, Loss: 9.182e-02
Iter: 120, Loss: 8.464e-02
Iter: 130, Loss: 8.438e-02
Iter: 140, Loss: 8.049e-02
Iter: 150, Loss: 7.429e-02
Iter: 160, Loss: 7.083e-02
Iter: 170, Loss: 9.098e-02
Iter: 180, Loss: 7.496e-02
Iter: 190, Loss: 7.073e-02
Iter: 200, Loss: 6.840e-02
Iter: 210, Loss: 6.681e-02
Iter: 220, Loss: 6.570e-02
Iter: 230, Loss: 6.505e-02
Iter: 240, Loss: 6.433e-02
Iter: 250, Loss: 6.417e-02
Iter: 260, Loss: 7.145e-02
Iter: 270, Loss: 6.276e-02
Iter: 280, Loss: 6.219e-02
Iter: 290, Loss: 6.149e-02
Iter: 300, Loss: 6.088e-02
Iter: 310, Loss: 6.037e-02
Iter: 320, Loss: 5.971e-02
Iter: 330, Loss: 5.917e-02
Iter: 340, Loss: 5.989e-02
Iter: 350, Loss: 6.396e-02
Iter: 360, Loss: 6.155e-02
Iter: 370, L

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)